In [19]:
import warnings
warnings.filterwarnings(action='ignore')
from functools import partial
from shapely import ops
import pyproj
import json, math
import matplotlib.pyplot as plt
from folium import plugins
import re
from matplotlib import font_manager, rc, rcParams
# def set_korea_font():
#     font_name = font_manager.FontProperties(fname="/System/Library/Fonts/Supplemental/AppleGothic.ttf").get_name()
#     rc('font', family=font_name)
#     rcParams.update({'font.size': 11})
#     rcParams['axes.unicode_minus'] = False  
# set_korea_font()
from shapely.geometry import Point as shapely_Point
from geopy.distance import great_circle as distance
from geopy.point import Point as Point
from math import sin, cos, atan2, sqrt, degrees, radians, pi

from geopy.distance import geodesic
import folium
from folium import plugins
import geopandas as gpd
import numpy as np
import pandas as pd
import shapely
import os
import datetime

#reproject import 필요

os.chdir('../../Feature/data')

def reproject(geom, from_proj=None, to_proj=None):
    tfm = partial(pyproj.transform, pyproj.Proj(init=from_proj), pyproj.Proj(init=to_proj))
    return ops.transform(tfm, geom)

#tab에서 찾기 편하게 모든 단어에 _count_ 붙음
#refer_data 는 data_set['function name']
#홍익선
overspeed_cam_count_ = pd.read_csv('overspeed_cam.csv')
floating_pop_count_ = pd.read_csv('floating_pop.csv')
child_pedestrian_count_ = pd.read_csv('child_pedestrian.csv')
bump_count_ = pd.read_csv('bump.csv')
parking_count_ = pd.read_csv('parking.csv')
parking_cctv_count_ = [parking_count_, pd.read_csv('parking_cctv.csv')]

#조민주
car_count_ = gpd.read_file("3.오산시_차량등록현황_격자.geojson", encoding = "utf-8")
child_count_ = gpd.read_file("child_count.geojson", encoding = "utf-8")
child_rate_count_ = gpd.read_file("child_rate.geojson", encoding = "utf-8")
elem_kinder_count_ = gpd.read_file("elem_kinder_count.csv", encoding = "utf-8")
isSchoolZone_count_ = pd.read_csv("9.오산시_어린이보호구역(re).csv", encoding = "utf-8")
numberSchoolZone_count_ = isSchoolZone_count_

#심재훈
accident_count_ = pd.read_csv("accident_count.csv")
chaos1_nearby_count_ = [gpd.read_file('23.오산시_상세도로망_LV6.geojson'),
                 pd.read_csv('chaos1.csv')]
width_nearby_count_ = gpd.read_file('23.오산시_상세도로망_LV6.geojson')
cross_road_nearby_count_ = width_nearby_count_

#이재훈
num_cram_school_count_ = pd.read_csv("학원.csv")
shortest_cross_count_ = gpd.read_file("횡단보도.json",encoding="utf-8")
shortest_traffic_signal_count_ = gpd.read_file('19.오산시_신호등.geojson',encoding="utf-8")
shortest_sidewalk_count_ =  gpd.read_file("인도.json",encoding="utf-8")


data_set = {
'overspeed_cam_count': overspeed_cam_count_,
'floating_pop_count':floating_pop_count_,
'child_pedestrian_count':child_pedestrian_count_,
'bump_count':bump_count_,
'parking_count':parking_count_,
'parking_cctv_count':parking_cctv_count_,
'car_count':car_count_,
'child_count':child_count_,
'child_rate_count':child_rate_count_,
'elem_kinder_count':elem_kinder_count_,
'isSchoolZone_count':isSchoolZone_count_,
'numberSchoolZone_count':numberSchoolZone_count_,
'accident_count':accident_count_,
'chaos1_nearby_count':chaos1_nearby_count_,
'width_nearby_count':width_nearby_count_,
'cross_road_nearby_count':cross_road_nearby_count_,
'num_cram_school_count':num_cram_school_count_,
'shortest_cross_count':shortest_cross_count_,
'shortest_traffic_signal_count':shortest_traffic_signal_count_,
'shortest_sidewalk_count':shortest_sidewalk_count_,
}

#only <= 12
data_set['accident_count'].loc[:,'가해운전자 연령'] = data_set['accident_count'].loc[:,'가해운전자 연령'].apply(lambda x: int(x[:-1]) if x[:-1].isnumeric() else 999)
data_set['accident_count'].loc[:,'피해운전자 연령'] = data_set['accident_count'].loc[:,'피해운전자 연령'].apply(lambda x: int(x[:-1]) if str(x)[:-1].isnumeric() else 999)

mask1 = data_set['accident_count'].loc[:,'가해운전자 연령'] <= 12
mask2 = data_set['accident_count'].loc[:,'피해운전자 연령'] <= 12

data_set['accident_count'] = data_set['accident_count'].loc[mask1|mask2]
data_set['accident_count'].index = range(len(data_set['accident_count']))

#tuple이 str으로 저장되는 문제
def _str_to_tuple(string):
  """
  input: multilinestring object
  output: arr_cords; lon, lat
  ex) multiline_to_arr_cords(road_gdf.loc[0].geometry)
  """
  return re.findall(r'\d+\.\d+',string)

def overspeed_cam_count(lat, lon, radi, refer_data, idx=False):
  """
  input: origin lat lon, radi(m)
  output: following point count
  optional: (idx of point, count)
  """
  if (type(refer_data['geometry'][0]) == str):
    refer_data['geometry'] = refer_data['geometry'].apply(_str_to_tuple)
  #apply
  bool_mask = refer_data['geometry'].apply(lambda x: geodesic((lat, lon),x).meters <= radi)
  if idx:
    return refer_data[bool_mask].index, sum(bool_mask)
  else:
    return sum(bool_mask)


def floating_pop_count(lat, lon, radi, refer_data, idx=False):
  """
  input: origin lat lon, radi(km)
  output: following point count
  optional: (idx of point, count)
  """
  if (type(refer_data['geometry'][0]) == str):
    refer_data['geometry'] = refer_data['geometry'].apply(_str_to_tuple)
  #apply
  bool_mask = refer_data['geometry'].apply(lambda x: geodesic((lat, lon), x).m <= radi)
  if idx:
    return refer_data[bool_mask].index, refer_data['mean'][bool_mask].mean()
  else:
    return refer_data['mean'][bool_mask].mean()

#어린이 보행/자전거 전체 사고 횟수
def child_pedestrian_count(lat, lon, radi, refer_data, typ = 'all', idx=False):
  """
  input: origin lat lon, radi(m), typ
  typ = 'all', '안전운전불이행','보행자보호의무위반', '신호위반'
  
  output: following point count
  optional: (idx of point, count)
  """
  if (type(refer_data['geometry'][0]) == str):
    refer_data['geometry'] = refer_data['geometry'].apply(_str_to_tuple)
  #apply
  if typ != 'all':
    refer_data = refer_data.loc[refer_data.법규위반==typ]
  bool_mask = refer_data['geometry'].apply(lambda x: geodesic((lat, lon),x).meters <= radi)
  if idx:
    return refer_data[bool_mask].index, sum(bool_mask)
  else:
    return sum(bool_mask)


def bump_count(lat, lon, radi, refer_data, idx = False):
    if (type(refer_data['geometry'][0]) == str):
        refer_data['geometry'] = refer_data['geometry'].apply(_str_to_tuple)
    dist = refer_data[['index','geometry']].apply(lambda x: (geodesic((lat, lon), x[1]).m, x[0]), axis = 1)
    dist = min(dist, key = lambda x :x[0])
                                        
    if idx:
        return dist
    else:
        return dist[0]


def parking_count(lat, lon, radi, refer_data, idx=False):
  """
  input: origin lat lon, radi(km)
  output: following point count
  optional: (idx of point, count)
  """
  if (type(refer_data['geometry'][0]) == str):
    refer_data['geometry'] = refer_data['geometry'].apply(_str_to_tuple)
  #apply
  bool_mask = refer_data['geometry'].apply(lambda x: geodesic((lat, lon),x).meters <= radi)
  if idx:
    return refer_data[bool_mask].index, sum(bool_mask)
  else:
    return sum(bool_mask)


def parking_cctv_count(lat, lon, radi, refer_data, idx=False):
  """
  cctv 1개당 평균 불법주정차 단속 건수
  input: origin lat lon, radi(km)
  output: following point count
  optional: (idx of point, count)
  """
  refer_data1, refer_data2 = refer_data[0], refer_data[1]
  if (type(refer_data1['geometry'][0]) == str):
    refer_data1['geometry'] = refer_data1['geometry'].apply(_str_to_tuple)
  if (type(refer_data2['geometry'][0]) == str):
    refer_data2['geometry'] = refer_data2['geometry'].apply(_str_to_tuple)
    
    
  #apply
  bool_mask1 = refer_data1['geometry'].apply(lambda x: geodesic((lat, lon),x).meters <= radi)
  bool_mask2 = refer_data2['geometry'].apply(lambda x: geodesic((lat, lon),x).meters <= radi)

  try:    
    result = sum(bool_mask1)/sum(bool_mask2)
  except:
    result = np.nan
  if idx:
    return refer_data1[bool_mask1].index, refer_data2[bool_mask2].index, result
  else:
    return result


#from https://medium.com/analytics-vidhya/calculating-distances-from-points-to-polygon-borders-in-python-a-paris-example-3b597e1ea291
def midpoint(a, b):
    a_lat, a_lon = radians(a.latitude), radians(a.longitude)
    b_lat, b_lon = radians(b.latitude), radians(b.longitude)
    delta_lon = b_lon - a_lon
    B_x = cos(b_lat) * cos(delta_lon)
    B_y = cos(b_lat) * sin(delta_lon)
    mid_lat = atan2(
        sin(a_lat) + sin(b_lat),
        sqrt(((cos(a_lat) + B_x) ** 2 + B_y ** 2))
    )
    mid_lon = a_lon + atan2(B_y, cos(a_lat) + B_x)
    # Normalise
    mid_lon = (mid_lon + 3 * pi) % (2 * pi) - pi

    return Point(latitude=degrees(mid_lat), longitude=degrees(mid_lon))

def get_line_midpoint(line):
    a = Point(line[0])
    b = Point(line[1])

    return midpoint(a,b)


def calculate_dist_to_line(line_a_lat, line_a_lng, line_b_lat, line_b_lng, point_object):
    a = Point(latitude=line_a_lat, longitude=line_a_lng)
    b = Point(latitude=line_b_lat, longitude=line_b_lng)
    dist = distance(midpoint(a, b), point_object)
    return dist


def get_min_distance_to_arr(arr_coords, point_object, unit='m'):
    min_dist = 999999
    line=[]
    for i, _ in enumerate(arr_coords):
        if i + 1 < len(arr_coords):
            dist = calculate_dist_to_line(
                line_a_lat=arr_coords[i][1],
                line_a_lng=arr_coords[i][0],
                line_b_lat=arr_coords[i + 1][1],
                line_b_lng=arr_coords[i + 1][0],
                point_object=point_object
            )
            if dist < min_dist:
                min_dist = dist
                line = [(arr_coords[i][1], arr_coords[i][0]), (arr_coords[i + 1][1], arr_coords[i + 1][0])]
        else:
            dist = calculate_dist_to_line(
                line_a_lat=arr_coords[i][1],
                line_a_lng=arr_coords[i][0],
                line_b_lat=arr_coords[0][1],
                line_b_lng=arr_coords[0][0],
                point_object=point_object
            )
            if dist < min_dist:
                min_dist = dist
                line = [(arr_coords[i][1], arr_coords[i][0]), (arr_coords[0][1], arr_coords[0][0])]

    if unit == 'm':
        return min_dist.m, line
    elif unit == 'km':
        return min_dist.km, line
    else:
        return min_dist, line
    
#additional interrial function
def _multiline_to_arr_cords(mls):
  """
  input: multilinestring object
  output: arr_cords; lon, lat
  ex) multiline_to_arr_cords(road_gdf.loc[0].geometry)
  """
  arr_cords = []
  lon_lat = re.findall(r'\d+\.\d+',str(mls))
  for i in range(0,len(lon_lat),2):
    arr_cords.append((float(lon_lat[i]),float(lon_lat[i+1])))
  return arr_cords

#(lat, lon), (lon, lat) 자꾸 갈려서 나오네
def _shPoint_to_list(shpoint):
  c1, c2 = re.findall(r'\d+\.\d+',str(shpoint))
  if c1 < c2:
    lon, lat = float(c1), float(c2)
  else:
    lat, lon = float(c1), float(c2)
  return lat, lon

def _list_to_gpPoint(mylist):
  return Point(latitude=mylist[0], longitude=mylist[1])

def _shPoint_to_gpPoint(shpoint):
  mylist = _shPoint_to_list(shpoint)
  return Point(latitude=mylist[0], longitude=mylist[1])


def road_nearby(lat, lon, radi, data):
  """
  input: origin lat lon, radi(m), data(shapely multilinestring)
  change in type: shapely(Euclidian) -> list -> geopy(Great Circle)
  #watchout for (lat lon), (lon lat)
  output: following roads
  
  #test_set
  lat, lon = (37.14875860564099, 127.0773701360968)
  radi = 1
  road_nearby(lat, lon, radi, road_gdf)
  """
  road_gdf = data
  dist_array = road_gdf.geometry.apply(lambda x: get_min_distance_to_arr(_multiline_to_arr_cords(x), _shPoint_to_gpPoint((lat, lon)), unit='m')[0])
  result = road_gdf.loc[dist_array < radi,:]
  while (result.size == 0):
    radi += 5
    result = road_gdf.loc[dist_array < radi,:]
  return result

def child_count(lat, lon, radi, refer_data):
    refer_data.val[refer_data.val.isna()] = 0
    distance_arr = refer_data.geometry.apply(lambda x : get_min_distance_to_arr
                    (_multiline_to_arr_cords(x),
                     _shPoint_to_gpPoint((lat, lon)), unit = 'm')[0])
    sum_count = refer_data[distance_arr <= radi]["val"].max()
    return sum_count


def child_rate_count(lat, lon, radi, refer_data):
    distance_arr = refer_data.geometry.apply(lambda x : get_min_distance_to_arr
                    (_multiline_to_arr_cords(x),
                     _shPoint_to_gpPoint((lat, lon)), unit = 'm')[0])
    selected_data = refer_data[distance_arr <= radi]
    child_rate = sum(selected_data["val"])/sum(selected_data["total"])
    return child_rate



def elem_kinder_count(lat, lon, radi, refer_data, typ = 'all'):
    """
    typ = 'all', '유치원과 어린이집', '초등학교'
    """
    if typ != 'all':
        refer_data = refer_data.loc[refer_data.시설구분==typ]
    data_loc = pd.Series(list(zip(refer_data["시설위치_위도"], refer_data["시설위치_경도"])))
    count_sum = data_loc.apply(lambda x: geodesic((lat, lon),x).m <= radi).sum()
    return count_sum

def accident_count(lat, lon, radi, refer_data, idx=False):
  """
  input: origin lat lon, radi(m), data(accidents record)
  output: following point count
  optional: (idx of point, count)
  """

  if (type(refer_data['geometry'][0]) == str):
    refer_data['geometry'] = refer_data['geometry'].apply(_str_to_tuple)
  #apply
  bool_mask = refer_data['geometry'].apply(lambda x: geodesic((lat, lon),x).m <= radi)
  if idx:
    return refer_data[bool_mask].index, sum(bool_mask)
  else:
    return sum(bool_mask)


def chaos1_nearby_count(lat, lon, radi, refer_data):
  """
  input: origin lat lon, radi(m), road_data(shapely multilinestring), chaos1_data
  output: following road's chaos1 by different times or lanes
  
  #test_set
  lat, lon = (37.14875860564099, 127.0773701360968)
  radi = 1
  chaos1_nearby(lat, lon, radi, road_gdf, chaos1_gdf)[11]
  #link_id 끝자리 01(상행) 02(하행) 순으로 나옴
  평일 주말 총 평균값
   """
  road_data, chaos1_data = refer_data[0], refer_data[1]
  roads = road_nearby(lat, lon, radi, road_data)
  result = roads.link_id.apply(lambda x: chaos1_data.loc[(chaos1_data.loc[:,'link_id']//100)==int(x),['time_range','chaos1']].mean()[0])
  return result.mean()



def width_nearby_count(lat, lon, radi, road_data):
  roads = road_nearby(lat, lon, radi, road_data)
  return roads.width.apply(int).mean()

def cross_road_nearby_count(lat, lon, radi, road_data):
  """
  4	교차로 통로
  32	복합교차로
  64	로타리
  128	회전교차로

  output = # of cross roads
  """
  cross_roads = [4,32,64,128]
  roads = road_nearby(lat, lon, radi, road_data)
  result = roads.loc[roads.link_type.apply(lambda x: int(x) in cross_roads)]
  return len(result)

def road_nearby(lat, lon, radi, data):
  """
  input: origin lat lon, radi(m), data(shapely multilinestring)
  change in type: shapely(Euclidian) -> list -> geopy(Great Circle)
  #watchout for (lat lon), (lon lat)
  output: following roads
  
  #test_set
  lat, lon = (37.14875860564099, 127.0773701360968)
  radi = 1
  road_nearby(lat, lon, radi, road_gdf)
  """
  road_gdf = data
  dist_array = road_gdf.geometry.apply(lambda x: get_min_distance_to_arr(_multiline_to_arr_cords(x), _shPoint_to_gpPoint((lat, lon)), unit='m')[0])
  result = road_gdf.loc[dist_array < radi,:]
  while (result.size == 0):
    radi += 5
    result = road_gdf.loc[dist_array < radi,:]
  return result


def num_cram_school_count(lat, lon, radi, refer_data):
    cram_school = pd.Series(list(zip(refer_data["시설위치_위도"], refer_data["시설위치_경도"])))
    return cram_school.apply(lambda x: geodesic((lat, lon),x).m <= radi).sum()


#reproject 후 polygon 그대로
def shortest_cross_count(lat, lon, radi, refer_data, idx = False):
    polygon1 = reproject(shapely_Point(lon,lat), 'EPSG:4326', 'EPSG:26944')
        
    result = refer_data[["OBJECTID","geometry"]].apply(lambda x : (x[1].distance(polygon1), x[0]), axis = 1)
    result = min(result, key = lambda x :x[0])
    if idx:
        return result
    return result[0]


def shortest_traffic_signal_count(lat, lon, radi, refer_data, idx = False):
    result = refer_data[["OBJECTID","geometry"]].apply(lambda x : (geodesic((lat, lon),(x[1].y,x[1].x)).m, x[0]), axis = 1)
    result = min(result, key = lambda x :x[0])
    if idx:
        return result
    return result[0]

def shortest_sidewalk_count(lat, lon, radi, refer_data):
    polygon1 = reproject(shapely_Point(lon,lat), 'EPSG:4326', 'EPSG:26944')
    result = refer_data["geometry"].apply(lambda x : x.distance(polygon1))
    result = min(result)
    return result

In [ ]:
# 아래 내용부터만 추가하면 됨, 위에건 그냥 불러오기

In [45]:
def accident_nearby(lat, lon, radi, refer_data, idx=False):
  
  if (type(refer_data['geometry'][0]) == str):
    refer_data['geometry'] = refer_data['geometry'].apply(_str_to_tuple)
  #apply
  bool_mask = refer_data['geometry'].apply(lambda x: geodesic((lat, lon),x).m <= radi)

  return refer_data[bool_mask]

In [21]:
accidents = pd.read_csv('./accident_count.csv').iloc[:,2:]
danger_loc = pd.read_csv('../../이재훈 model 최종/data/result_danger_loc(non-school).csv')

In [22]:
def make_info(latlon):
  lat, lon = latlon
  df1 = pd.DataFrame([[-1],[-1],[-1],[-1],[-1],[-1],[-1],[-1]]).T
  df1.columns = ['overspeed_cam_count','floating_pop_count','child_count','elem_kinder_count','width_nearby_count',
                 'num_cram_school_count','accident_type','law_break']
  df1['overspeed_cam_count'] = overspeed_cam_count(lat, lon, 100,data_set['overspeed_cam_count'])
  df1['floating_pop_count'] = floating_pop_count(lat, lon, 50,data_set['floating_pop_count'])
  df1['child_count'] = child_count(lat, lon, 1000,data_set['child_count'])
  df1['elem_kinder_count'] = elem_kinder_count(lat, lon, 400,data_set['elem_kinder_count'])
  df1['width_nearby_count'] = width_nearby_count(lat, lon, 1,data_set['width_nearby_count'])
  df1['num_cram_school_count'] = num_cram_school_count(lat, lon, 400,data_set['num_cram_school_count'])
  df1['accident_type'] = str(list(accident_nearby(lat, lon, 15, accidents).loc[:,'사고유형']))
  df1['law_break'] = str(list(accident_nearby(lat, lon, 15, accidents).loc[:,'법규위반']))
  return df1.loc[0]

In [46]:
df1 = danger_loc.iloc[:,1:3].apply(make_info,axis=1)
explain = pd.read_csv('../../이재훈 model 최종/data/특이사항_추천.csv',header=None)
explain.columns = ['특이사항','추천시설물']
pd.concat((df1,explain),axis=1).to_csv('../../이재훈 model 최종/data/비스쿨존 정리.csv')

In [47]:
pd.concat((df1,explain),axis=1)

,overspeed_cam_count,floating_pop_count,child_count,elem_kinder_count,width_nearby_count,num_cram_school_count,accident_type,law_break,특이사항,추천시설물
0,0,34.715833,182.0,7,3.00,189,"['차대차 - 측면충돌', '차대차 - 측면직각충돌']","['신호위반', '기타']",근린공원 및 주거시설 근처/산업도로 가는길 인근(속도 높을것으로 예상됨)/관공서 근...,주정차 단속 카메라
1,0,528.720208,182.0,9,4.00,206,['차대사람 - 횡단중'],['안전운전불이행'],근린공원 및 주거시설,시야방해 홍보물 단속(우회전시)
2,0,42.457500,182.0,9,4.00,0,[],[],사고 날만한게 없음/ 주택가 인근,주택가 근처 감속 표지판
3,0,186.142500,277.0,14,2.00,87,"['차대사람 - 횡단중', '차대사람 - 기타']","['안전운전불이행', '안전운전불이행']",주택가 클러스터 정중앙/ 많은 주정차 좁은 차도,과속방지턱/ 주정차 단속
4,0,390.898125,24.0,1,1.00,17,['차대사람 - 기타'],['안전운전불이행'],감속 안해도 우회전 할 수 있는 구조/ 과속이나 겨울철 미끄럼방지 (코너링이 미끄러...,코너에 과속방지턱/ 미끄럼 방지 포장
5,0,0.614792,258.0,0,2.00,0,['차대차 - 기타'],['안전운전불이행'],유동인구가 없는 도로여서 도로 환경 매우 열악,삼거리 진입전 과속방지턱/ 도로 안전선 페인팅 다시/ 삼거리 진입전 정지선 페인팅
6,0,54.968750,126.0,4,2.00,8,[],[],생활도로구역/ 2차선임에도 주정차가 많음(시야방해)/ 횡단보도와 거리가 멀고 감속 ...,가드펜스/ 주정차 단속
7,0,79.515833,126.0,0,4.00,0,[],[],매우 넓은 4거리,횡단보도 섬
8,0,64.133611,422.0,7,2.00,25,"['차대차 - 정면충돌', '차대차 - 기타', '차대차 - 측면직각충돌', '차대...","['신호위반', '신호위반', '신호위반', '신호위반']",신호 받으려고 달리다가 늦게 멈춰서 횡단보도에 걸치는 경우 많아 보임/ 매우 혼잡한...,과속방지카메라
9,0,358.189375,162.0,12,3.00,213,['차대사람 - 차도통행중'],['안전운전불이행'],다음 횡단보도까지 거리가 멀다,가드펜스
